### Test connection with GraphDB

In [5]:
from langchain_community.graphs import Neo4jGraph

In [6]:
graph = Neo4jGraph(url = 'bolt://localhost:7687',
                   username = 'neo4j',
                   password = '12345678',
                   database = 'neo4j')

In [7]:
# Match all nodes in the graph
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = graph.query(cypher)
result

In [8]:
graph.query("""
        MATCH (mergedChunk:Chunk {chunkSeqId: $value})
        RETURN mergedChunk
        ORDER BY rand()
        LIMIT 1
         """,
params={"value": 1})
     

In [53]:
### Loadiing the data into Database

In [9]:
import pandas as pd
from pyprojroot import here

In [39]:
df = pd.read_csv("/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/movie.csv")
df.head(5)

In [11]:
df.shape

In [12]:
df2 = pd.read_csv('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/data 2.csv')
df2.head(5)

In [13]:
df2 = pd.read_csv('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/data 2.csv')
df2.head(5)

In [14]:
df2.head(5)

In [15]:
from pyprojroot import here

movie_csv_path = here('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/movie.csv')

In [16]:
df2.tail(5)

In [17]:
#df['location'] = df2['tagline ']

In [18]:
df.columns

In [31]:
!pwd

In [32]:
pd.read_csv('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/movies1.csv')

In [30]:
df.to_csv('movies1.csv')

In [19]:
## Connection with LLM

In [20]:
from langchain.llms import AI21

In [40]:
df.columns

In [41]:
graph.query("""
LOAD CSV WITH HEADERS FROM 'file:///' + $movie_directory       // Load the data from csv file $movie_directory
AS row                                                         //Each row in the csv file represent as 'row'
    
MERGE (m:Movie {id:row.movieId} )
Set m.released = date(row.released),
    m.title = row.title,
    m.tagline = row.tagline,
    m.imdbRating = toFloat(row.imdbRating)
    
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
    
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
    
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:GENRE {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
    
MERGE (l:Location {name:trim(row.location)})
MERGE (m)-[:WAS_TAKEN_IN]->(l)

MERGE (s:SimilarMovie {name:trim(row.similar_movie)})
MERGE (m)-[:IS_SIMILAR_TO]->(s)
""",
params = {"movie_directory": str('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/movies1.csv')} )

In [22]:
str('/Users/mohitverma/Documents/RAG Chatbots/Rag-chatbot graphDB/movie.csv')